In [1]:
#| default_exp trait

In [2]:
#| export

from fastcore.test import test_eq, test_close, test_fail
import jax
import jax.numpy as jnp
from flax.struct import dataclass as flax_dataclass

from typing import List, Optional
from jaxtyping import Array, Float, Int, PyTree

from chewc.structs import Population, SimParam
from chewc.popgen import quick_haplo

In [3]:
#| export

# chewc/trait.py
from typing import Optional, Tuple, Union

import jax
import jax.numpy as jnp
from flax.struct import dataclass as flax_dataclass
from functools import partial

# Import the necessary structures from your existing codebase
from chewc.structs import Population, SimParam
from chewc.kernels import compute_dosage

# --- Base Trait Structure ---

@flax_dataclass(frozen=True)
class TraitCollection:
    """
    A collection of additive genetic traits.
    """
    loci_loc: jnp.ndarray  # Shape: (n_qtl,)
    add_eff: jnp.ndarray   # Shape: (n_qtl, n_traits)
    intercept: jnp.ndarray # Shape: (n_traits,)

    @property
    def n_traits(self) -> int:
        return self.intercept.shape[0]

    @property
    def n_loci(self) -> int:
        return self.loci_loc.shape[0]

# --- JIT-Compatible Kernels for Trait Calculation ---

def _calculate_gvs_from_dosage(
    dosage: jnp.ndarray,
    traits: TraitCollection
) -> Tuple[jnp.ndarray, jnp.ndarray]:
    """
    Calculates genetic values from dosage using efficient matrix multiplication.
    """
    qtl_dosage = dosage[:, traits.loci_loc]
    all_bv = qtl_dosage @ traits.add_eff
    all_gvs = all_bv + traits.intercept
    return all_bv, all_gvs


def set_pheno_h2(
    key: jax.random.PRNGKey,
    pop: Population,
    sp: SimParam,
    h2: Union[float, jnp.ndarray],
    broad_sense: bool = False,
) -> Population:
    """
    Computes and sets phenotypes based on a target heritability.
    This function is now fully JIT-compatible.
    """
    if sp.traits is None:
        raise ValueError("SimParam object must have a TraitCollection to set phenotypes.")

    dosage = compute_dosage(pop.geno, pop.is_active)
    bv, gv = _calculate_gvs_from_dosage(dosage, sp.traits)

    active_mask = pop.is_active

    # --- FIX WAS HERE ---
    # Boolean array indexing `gv[active_mask]` is not JIT-compatible because the
    # output shape is not static.
    # The idiomatic JAX way to compute a masked variance is to use the `where` argument.
    # The mask must be broadcastable to the array's shape.
    mask_for_traits = active_mask[:, None]

    if broad_sense:
        genetic_variance = jnp.var(gv, axis=0, where=mask_for_traits)
    else:
        genetic_variance = jnp.var(bv, axis=0, where=mask_for_traits)

    h2 = jnp.asarray(h2)
    var_e = genetic_variance * ((1 / (h2 + 1e-8)) - 1)
    var_e = jnp.maximum(0, var_e)

    environmental_noise = jax.random.normal(key, gv.shape) * jnp.sqrt(var_e)
    pheno = gv + environmental_noise

    bv = jnp.where(pop.is_active[:, None], bv, jnp.nan)
    gv = jnp.where(pop.is_active[:, None], gv, jnp.nan)
    pheno = jnp.where(pop.is_active[:, None], pheno, jnp.nan)

    return pop.replace(bv=bv, gv=gv, pheno=pheno)


def set_pheno(
    key: jax.random.PRNGKey,
    pop: Population,
    sp: SimParam,
    var_e: jnp.ndarray
) -> Population:
    """
    Computes and sets phenotypes with a fixed environmental variance.
    """
    if sp.traits is None:
        raise ValueError("SimParam object must have a TraitCollection to set phenotypes.")

    dosage = compute_dosage(pop.geno, pop.is_active)
    bv, gv = _calculate_gvs_from_dosage(dosage, sp.traits)
    environmental_noise = jax.random.normal(key, gv.shape) * jnp.sqrt(var_e)
    pheno = gv + environmental_noise

    bv = jnp.where(pop.is_active[:, None], bv, jnp.nan)
    gv = jnp.where(pop.is_active[:, None], gv, jnp.nan)
    pheno = jnp.where(pop.is_active[:, None], pheno, jnp.nan)

    return pop.replace(bv=bv, gv=gv, pheno=pheno)


# --- Host-Side Helper Function to Add Traits ---

def add_trait_a(
    key: jax.random.PRNGKey,
    founder_pop: Population,
    sim_param: SimParam,
    n_qtl_per_chr: int,
    mean: jnp.ndarray,
    var: jnp.ndarray,
    cor_a: Optional[jnp.ndarray] = None,
    gamma: bool = False,
    shape: float = 1.0
) -> SimParam:
    """
    Adds one or more new additive traits to the simulation parameters.
    This function is run on the host, so boolean indexing is fine here.
    """
    n_loci_per_chr = sim_param.gen_map.shape[1]
    if n_qtl_per_chr > n_loci_per_chr:
        raise ValueError(
            f"Requested n_qtl_per_chr={n_qtl_per_chr}, but there are only "
            f"{n_loci_per_chr} loci available per chromosome."
        )

    n_traits = mean.shape[0]
    assert mean.shape == var.shape, "Mean and variance vectors must have the same shape."
    if cor_a is None:
        cor_a = jnp.identity(n_traits)

    key, sample_key, qtl_key, sign_key = jax.random.split(key, 4)

    n_total_qtl = n_qtl_per_chr * sim_param.gen_map.shape[0]
    all_loci_indices = jnp.arange(sim_param.gen_map.size)
    qtl_loc = jax.random.choice(qtl_key, all_loci_indices, shape=(n_total_qtl,), replace=False)
    qtl_loc = jnp.sort(qtl_loc)

    if gamma:
        gamma_effects = jax.random.gamma(sample_key, shape, shape=(n_total_qtl, n_traits))
        signs = jax.random.choice(sign_key, jnp.array([-1.0, 1.0]), shape=(n_total_qtl, n_traits))
        raw_effects = gamma_effects * signs
    else:
        raw_effects = jax.random.normal(sample_key, (n_total_qtl, n_traits))

    cholesky_matrix = jnp.linalg.cholesky(cor_a)
    correlated_raw_effects = raw_effects @ cholesky_matrix.T

    temp_traits = TraitCollection(
        loci_loc=qtl_loc,
        add_eff=correlated_raw_effects,
        intercept=jnp.zeros(n_traits)
    )

    founder_dosage = compute_dosage(founder_pop.geno, founder_pop.is_active)
    initial_bvs, initial_gvs = _calculate_gvs_from_dosage(founder_dosage, temp_traits)

    active_mask = founder_pop.is_active
    initial_vars = jnp.var(initial_gvs[active_mask], axis=0)
    initial_means = jnp.mean(initial_bvs[active_mask], axis=0)

    scaling_factors = jnp.sqrt(var / (initial_vars + 1e-8))
    final_intercepts = mean - (initial_means * scaling_factors)
    final_add_eff = correlated_raw_effects * scaling_factors

    trait_collection = TraitCollection(
        loci_loc=qtl_loc,
        add_eff=final_add_eff,
        intercept=final_intercepts
    )

    return sim_param.replace(traits=trait_collection)



In [4]:
#| hide
import nbdev; nbdev.nbdev_export()